# Final Homework Assignment: Bayesian Inverse Problem for Spring-Slider Earthquake Model

## Overview

This assignment focuses on solving a **Bayesian inverse problem** for a physics-based earthquake model using advanced sampling techniques. You will generate synthetic observational data from a spring-slider model with rate-state friction, then invert the data to recover the unknown model parameters using one of three state-of-the-art inverse methods.


---

## Problem Description

### The Forward Model: Spring-Slider with Rate-State Friction

The spring-slider model simulates fault slip evolution after a sudden stress change. The model is governed by rate-and-state friction laws and includes the following **7 unknown parameters**:

| Parameter | Symbol | Physical Meaning | Typical Range |
|-----------|--------|------------------|---------------|
| Critical displacement | $d_c$ | State evolution length scale | 10⁻⁵ - 10⁻³ m |
| Rate parameter A | $A$ | Direct velocity effect | 0.005 - 0.015 |
| Rate parameter B | $B$ | Evolution effect | 0.003 - 0.012 |
| Normal stress | $\sigma$ | Effective normal stress on fault | 10 - 50 MPa |
| Slip patch radius | $L$ | Characteristic length scale | 10,000 - 50,000 m |
| Load-point velocity | $v_{\infty}$ | Tectonic loading rate | 0.01 - 0.1 m/yr |
| Stress change | $\Delta\tau$ | Instantaneous stress perturbation | 0.5 - 5.0 MPa |

### Governing Equations

The model solves the following coupled differential equations:

**Velocity evolution:**
$$\frac{dv}{dt} = \frac{\frac{\dot{\tau}}{\sigma} - \frac{B}{\theta}\frac{d\theta}{dt}}{\frac{\eta}{\sigma} + \frac{A}{v}}$$

**State variable (Aging Law):**
$$\frac{d\theta}{dt} = 1 - \frac{\theta v}{d_c}$$

**Shear stress:**
$$\frac{d\tau}{dt} = k(v_{\infty} - v)$$

**Displacement:**
$$\frac{du}{dt} = v$$

where $\eta = \mu/(2\beta L)$ is the radiation damping coefficient.

### Observable Data

The forward model produces a **displacement time series** $u(t)$ showing the evolution of fault slip following the stress change. This is what you will observe (with noise) and use to infer the parameters.

---

## Part 1: Generate Synthetic Data 

### Task 1.1: Set Up the Forward Model

1. Navigate to the `spring_slider/` folder in this directory
2. Open `setup_spring_slider.ipynb`
3. Locate the cell that generates synthetic data (the last cell in the notebook)

### Task 1.2: Choose True Parameters

Select "true" parameter values for your synthetic data generation. These should be:
- Within physically reasonable ranges (see table above)
- **Record these values** - they are what you'll try to recover!
- Modify the parameter values in the synthetic data generation cell

**True parameter set:**
```python
d_c = 1e-4          # critical displacement (m)
A = 0.01            # rate-state parameter A
B = 0.006           # rate-state parameter B
sigma = 20          # normal stress (MPa)
L = 25000           # radius of slip patch (m)
v_inf = 0.03        # load-point velocity (m/yr)
deltau = 0.7        # instantaneous stress change (MPa)
```

Feel free to choose different values, but **document your choices clearly**.

### Task 1.3: Generate Noisy Observations

1. Run the synthetic data generation cell in `setup_spring_slider.ipynb`
2. The script will create a file: `synthetic_slip_data.txt` with columns:
   - Time (years)
   - Fault slip (meters)  
   - Sigma (measurement uncertainty in meters)

3. The noise level is set in the notebook (default 5% of signal amplitude)
4. Verify the data file was created successfully

### Task 1.4: Load and Visualize Data

In your homework notebook:
1. Load the synthetic data from `synthetic_slip_data.txt`
2. Create a plot showing:
   - Observed displacement vs. time
   - Error bars (± 1σ)
   - Indicate the noise level used



---

## Part 2: Formulate the Inverse Problem 

### Task 2.1: Define the Forward Model Function

Create a Python function that:
1. Takes the 7 model parameters as input
2. Calls the spring-slider forward model (from `spring_slider_model.py`)
3. Returns predicted displacement at the observation times


**Function signature:**
```python
def forward_model(params, times):
    """
    Run spring-slider forward model
    
    Parameters:
    -----------
    params : array-like, length 7
        [d_c, A, B, sigma, L, v_inf, deltau]
    times : array
        Observation times (years)
    
    Returns:
    --------
    u_pred : array
        Predicted displacement (meters)
    """
    # Your implementation here
```

### Task 2.2: Define the Likelihood Function

Assuming Gaussian observation errors:

$$p(\mathbf{d}|\mathbf{m}) = \prod_{i=1}^{N} \frac{1}{\sqrt{2\pi\sigma_i^2}} \exp\left(-\frac{(d_i - g_i(\mathbf{m}))^2}{2\sigma_i^2}\right)$$

where:
- $\mathbf{d}$ = observed data
- $\mathbf{m}$ = model parameters (the 7 unknowns)
- $g_i(\mathbf{m})$ = forward model prediction at time $t_i$
- $\sigma_i$ = measurement uncertainty at time $t_i$

**Log-likelihood:**
$$\log p(\mathbf{d}|\mathbf{m}) =  - \frac{1}{2}\sum_{i=1}^{N}\frac{(d_i - g_i(\mathbf{m}))^2}{\sigma_i^2}$$

Implement this as:
```python
def log_likelihood(params, data, times, sigma):
    """
    Compute log-likelihood for spring-slider model
    
    Parameters:
    -----------
    params : array, length 7
        Model parameters
    data : array
        Observed displacements
    times : array
        Observation times
    sigma : array
        Measurement uncertainties
    
    Returns:
    --------
    log_L : float
        Log-likelihood value
    """
    # Your implementation here
```

### Task 2.3: Define Prior Distributions

Choose prior distributions for each parameter. Consider:
- **Uniform priors**: $p(m_i) = 1/(b_i - a_i)$ for $m_i \in [a_i, b_i]$
- **Log-uniform priors**: For parameters spanning orders of magnitude (e.g., $d_c$)
- **Physical constraints**: Ensure $A < B$ for velocity-weakening behavior

**Suggested priors:**

| Parameter | Prior Type | Range/Parameters |
|-----------|------------|------------------|
| $d_c$ | Log-uniform | $[10^{-5}, 10^{-3}]$ m |
| $A$ | Uniform | $[0.005, 0.015]$ |
| $B$ | Uniform | $[0.003, 0.012]$ |
| $\sigma$ | Uniform | $[10, 50]$ MPa |
| $L$ | Uniform | $[10000, 50000]$ m |
| $v_{\infty}$ | Uniform | $[0.01, 0.1]$ m/yr |
| $\Delta\tau$ | Uniform | $[0.5, 5.0]$ MPa |

Implement:
```python
def log_prior(params):
    """
    Compute log-prior probability
    
    Parameters:
    -----------
    params : array, length 7
        Model parameters
    
    Returns:
    --------
    log_p : float
        Log-prior value (or -inf if outside bounds)
    """
    # Your implementation here
```

### Task 2.4: Define Posterior

Combine likelihood and prior:

$$p(\mathbf{m}|\mathbf{d}) \propto p(\mathbf{d}|\mathbf{m}) \cdot p(\mathbf{m})$$

$$\log p(\mathbf{m}|\mathbf{d}) = \log p(\mathbf{d}|\mathbf{m}) + \log p(\mathbf{m}) + \text{const}$$


---

## Part 3: Choose and Implement Inverse Method (40 points)

**Select ONE of the following three methods for your inversion.** Each method has different strengths and computational characteristics.

---

### Option 1: Adaptive Metropolis MCMC

**Description:** Standard Metropolis-Hastings with adaptive proposal covariance to improve acceptance rates and mixing.

**Algorithm:**
1. Start with initial parameter guess $\mathbf{m}_0$
2. Use initial proposal covariance (e.g., diagonal with parameter-specific scales)
3. During burn-in, adapt proposal covariance based on sample history
4. After burn-in, freeze covariance and continue sampling

**Key Implementation Steps:**

```python
def adaptive_metropolis_mcmc(log_posterior, initial_params, n_iterations, 
                             adapt_interval=100, adapt_until=5000):
    """
    Adaptive Metropolis MCMC sampler
    
    Parameters:
    -----------
    log_posterior : function
        Function that computes log-posterior
    initial_params : array
        Starting parameter values
    n_iterations : int
        Total number of iterations
    adapt_interval : int
        How often to update proposal covariance
    adapt_until : int
        Iteration after which to stop adapting
    
    Returns:
    --------
    samples : array (n_iterations, n_params)
        MCMC samples
    acceptance_rate : float
        Overall acceptance rate
    """
    # Your implementation
```

**Requirements:**
- Implement proposal adaptation using sample covariance
- Target acceptance rate: 20-40%
- Track acceptance rate, adapt proposal accordingly

**Tuning parameters to optimize:**
- Initial proposal scale
- Adaptation interval
- When to stop adapting


---

### Option 2: Exchange Monte Carlo (Parallel Tempering)

**Description:** Run multiple chains at different "temperatures" that can exchange states, improving exploration of multi-modal posteriors.

**Algorithm:**
1. Define temperature ladder: $T_1 = 1, T_2, ..., T_N$ (e.g., $N = 5-10$ chains)
2. Run parallel chains sampling from $p(\mathbf{m}|\mathbf{d})^{1/T_i}$
3. Periodically propose exchanges between adjacent chains
4. Only the cold chain ($T_1 = 1$) samples the true posterior

**Key Implementation Steps:**

```python
def exchange_monte_carlo(log_posterior, initial_params, temperatures, 
                        n_iterations, exchange_interval=10):
    """
    Parallel tempering / Exchange Monte Carlo
    
    Parameters:
    -----------
    log_posterior : function
        Function that computes log-posterior
    initial_params : array or list of arrays
        Starting parameters for each chain
    temperatures : array
        Temperature ladder (T[0] = 1.0 for cold chain)
    n_iterations : int
        Number of iterations
    exchange_interval : int
        How often to attempt exchanges
    
    Returns:
    --------
    cold_chain_samples : array
        Samples from cold chain (the target distribution)
    all_chains : list
        Samples from all chains (for diagnostics)
    exchange_rates : array
        Exchange acceptance rates between adjacent chains
    """
    # Your implementation
```

**Requirements:**
- Implement 5-8 parallel chains
- Temperature ladder with geometric spacing
- Exchange moves every 10-50 iterations
- Target exchange rates: 20-40% between adjacent chains
- Only analyze cold chain ($T=1$) for parameter inference

**Tuning parameters to optimize:**
- Temperature ladder (affects exchange rates)
- Number of chains
- Exchange frequency



---

### Option 3: Reduced-Order Model (ROM) with MCMC

**Description:** Build a reduced-order surrogate model to accelerate the forward model, then apply standard Metropolis sampling.

**Algorithm:**
1. **Training phase**: Generate database of forward model runs
2. **ROM construction**: Use iPOD 
3. **MCMC sampling**: Use ROM instead of full model in likelihood evaluation

**Key Implementation Steps:**

**Step 3.1: Build ROM Database**
```python
def build_rom_database(n_training_points=100):
    """
    Generate training data for reduced-order model
    
    Steps:
    1. Sample parameter space (e.g., Latin Hypercube Sampling)
    2. Run forward model at each training point
    3. Store (parameters, output) pairs
    
    Returns:
    --------
    training_params : array (n_training, 7)
        Parameter sets used for training
    training_outputs : array (n_training, n_times)
        Forward model outputs
    """
    # Your implementation
```

**Step 3.2: Train ROM**

Options for ROM:
- **Polynomial regression**: $u(t, \mathbf{m}) \approx \sum_{i} \alpha_i(\mathbf{m}) \phi_i(t)$
- **Gaussian Process Regression**: Interpolate between training points
- **Radial Basis Functions**: Fast interpolation scheme
- **POD-based**: Project onto dominant modes

```python
def train_rom(training_params, training_outputs):
    """
    Train reduced-order model
    
    Returns:
    --------
    rom_predictor : function or object
        Trained ROM that can predict u(t, m) quickly
    """
    # Your implementation
```

**Step 3.3: MCMC with ROM**
```python
def mcmc_with_rom(rom_predictor, data, times, sigma, 
                  n_iterations=50000):
    """
    Metropolis MCMC using ROM for forward model evaluations
    
    Parameters:
    -----------
    rom_predictor : function
        Trained ROM that predicts displacement given parameters
    
    Returns:
    --------
    samples : array
        MCMC samples
    """
    # Your implementation
```

**Requirements:**
- Training database: 100-500(?) forward model runs
- ROM must achieve < 10% prediction error on validation set


**Tuning parameters to optimize:**
- Number of training points
- ROM type and hyperparameters
- Training/validation split


---

### General Requirements for All Methods

Regardless of which method you choose:

1. **Convergence diagnostics:**
   - Trace plots for all parameters
   - Autocorrelation analysis


2. **Parameter estimates:**
   - Posterior means and standard deviations
   - 95% credible intervals
   - Marginal posterior distributions (histograms)
   - pairwise correlations

